# Tiền xử lí dữ liệu

In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import cv2
import tensorflow as tf
from PIL import Image
import os
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from keras.models import Sequential, load_model
from keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout

data = []
labels = []
classes = 43
cur_path = os.getcwd()

#Retrieving the images and their labels 
for i in range(classes):
    path = os.path.join(cur_path,'train',str(i))
    images = os.listdir(path)

    for a in images:
        try:
            image = Image.open(path + '\\'+ a)
            image = image.resize((30,30))
            image = np.array(image)
            #sim = Image.fromarray(image)
            data.append(image)
            labels.append(i)
        except:
            print("Error loading image")

#Converting lists into numpy arrays
data = np.array(data)
labels = np.array(labels)

In [2]:
print(data.shape, labels.shape)
#Splitting training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)

print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

#Converting the labels into one hot encoding
y_train = to_categorical(y_train, 43)
y_test = to_categorical(y_test, 43)

(39209, 30, 30, 3) (39209,)
(31367, 30, 30, 3) (7842, 30, 30, 3) (31367,) (7842,)


In [3]:
#Building the model
model = Sequential()
model.add(Conv2D(filters=32, kernel_size=(5,5), activation='relu', input_shape=X_train.shape[1:]))
model.add(Conv2D(filters=32, kernel_size=(5,5), activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(rate=0.25))
model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu'))
model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(rate=0.25))
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(rate=0.5))
model.add(Dense(43, activation='softmax'))

#Compilation of the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [4]:
epochs = 15
history = model.fit(X_train, y_train, batch_size=32, epochs=epochs, validation_data=(X_test, y_test))
model.save("my_model.h5")

Epoch 1/15
981/981 [==============================] - 39s 39ms/step - loss: 1.9994 - accuracy: 0.4970 - val_loss: 0.4095 - val_accuracy: 0.8930
Epoch 2/15
981/981 [==============================] - 34s 34ms/step - loss: 0.5638 - accuracy: 0.8321 - val_loss: 0.2177 - val_accuracy: 0.9546
Epoch 3/15
981/981 [==============================] - 31s 31ms/step - loss: 0.3417 - accuracy: 0.9002 - val_loss: 0.0854 - val_accuracy: 0.9782
Epoch 4/15
981/981 [==============================] - 30s 31ms/step - loss: 0.2887 - accuracy: 0.9176 - val_loss: 0.0710 - val_accuracy: 0.9819
Epoch 5/15
981/981 [==============================] - 31s 31ms/step - loss: 0.2567 - accuracy: 0.9268 - val_loss: 0.0543 - val_accuracy: 0.9864
Epoch 6/15
981/981 [==============================] - 38s 38ms/step - loss: 0.2361 - accuracy: 0.9341 - val_loss: 0.0770 - val_accuracy: 0.9804
Epoch 7/15
981/981 [==============================] - 36s 36ms/step - loss: 0.2240 - accuracy: 0.9377 - val_loss: 0.0474 - val_accuracy:

c:\Users\Tuan Anh\AppData\Local\Programs\Python\Python38\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
#plotting graphs for accuracy 
plt.figure(0)
plt.plot(history.history['accuracy'], label='training accuracy')
plt.plot(history.history['val_accuracy'], label='val accuracy')
plt.title('Accuracy')
plt.xlabel('epochs')
plt.ylabel('accuracy')
plt.legend()
plt.show()
 
plt.figure(1)
plt.plot(history.history['loss'], label='training loss')
plt.plot(history.history['val_loss'], label='val loss')
plt.title('Loss')
plt.xlabel('epochs')
plt.ylabel('loss')
plt.legend()
plt.show()

In [ ]:
#testing accuracy on test dataset
from sklearn.metrics import accuracy_score

# Đọc dữ liệu từ file CSV
y_test = pd.read_csv('Test.csv')

# Lấy các nhãn và đường dẫn ảnh
labels = y_test["ClassId"].values
imgs = y_test["Path"].values

data = []

# Xử lý ảnh
for img in imgs:
    image = Image.open(img)
    image = image.resize((30,30))
    data.append(np.array(image))

X_test = np.array(data)

# Dự đoán xác suất cho các lớp
pred = model.predict(X_test)

# Chuyển xác suất thành nhãn lớp
pred_classes = np.argmax(pred, axis=1)

# Tính độ chính xác với dữ liệu kiểm thử
print(accuracy_score(labels, pred_classes))

In [ ]:
model.save('traffic_classifier.h5')